## Evaluar el funcionamiento de la etapa 1 del proceso de optimización

In [ ]:

from plots.graph_room_outline import plot_outline_simple

from outline_optim import find_best_outline

# Inputs usuario
Lx = 2.5       # Largo de la sala en X 
Ly = 3         # Largo de la sala en Y
Lz = 2.2       # Alto de la sala
Dx = 0.4       # Delta X
Dy = 0.6       # Delta Y
Dz = 0.15      # Delta Z

## Posiciones fuente y receptor (en metros)
source_position = (1.9, 1.0, 1.3)
receptor_position = (1.25, 1.9, 1.2)

## Número de cortes en las paredes
n_walls = 2    

# Inicio del ciclo de iteración
## 0) Evalúa condiciones iniciales

## 1) Mejores dimensiones con geometrías simples
final_best_room, best_room_spacing, merit, mag_best = find_best_outline(Lx, Ly, Lz, Dx, Dy, Dz, source_position, receptor_position)
    
print("Espaciado en dX: ", best_room_spacing[0])
print("Espaciado en dY: ", best_room_spacing[1])
print(merit)

plot_outline_simple(Lx, Ly, Dx, Dy, final_best_room[0], final_best_room[1])


In [4]:
import numpy as np

def mutate_room(coords: np.ndarray,
                num_points: int = None,
                max_disp: float = 0.05,
                seed: int = None) -> np.ndarray:
    """
    Return a new coords array where either 2 or 4 points have been randomly
    displaced (within ±max_disp), preserving symmetry across the room's X-mirror plane,
    and ensuring no coordinate (x or y) goes negative.
    
    Parameters
    ----------
    coords : np.ndarray, shape (N, 2)
        Original room corner coordinates, assumed symmetric in X.
    num_points : int, optional
        Total number of points to move (must be 2 or 4). If None, choose randomly.
    max_disp : float, default=0.05
        Maximum absolute displacement (meters) along each axis.
    seed : int, optional
        Random seed for reproducibility.
    
    Returns
    -------
    new_coords : np.ndarray, shape (N, 2)
        The mutated coordinates, still symmetric in X and ≥ 0.
    """
    if seed is not None:
        np.random.seed(seed)
    
    N = len(coords)
    if N % 2 != 0:
        raise ValueError("coords length must be even for symmetric pairs.")
    
    # build (i, j) mirror-pair list
    pairs = [(i, N-1-i) for i in range(N//2)]
    
    # decide how many points to move
    if num_points is None:
        num_points = np.random.choice([2, 4])
    if num_points not in (2, 4):
        raise ValueError("num_points must be 2 or 4")
    num_pairs = num_points // 2
    
    # copy coords so original is unchanged
    new_coords = coords.astype(float).copy()
    
    # compute symmetry plane X = center_x from first pair
    i0, j0 = pairs[0]
    center_x = 0.5 * (coords[i0,0] + coords[j0,0])
    
    # pick which mirror-pairs to perturb
    chosen_pairs = np.random.choice(len(pairs), size=num_pairs, replace=False)
    
    for idx in chosen_pairs:
        i, j = pairs[idx]
        
        # allowed dx range so that new_x_i ≥ 0 and its mirror ≥ 0
        x_i, y_i = coords[i]
        dx_min = max(-max_disp,   -x_i)                       # prevents x_i + dx < 0
        dx_max = min( max_disp, (2*center_x - x_i) )         # prevents mirrored x < 0
        
        # allowed dy range so that new_y_i ≥ 0
        dy_min = max(-max_disp, -y_i)
        dy_max = max_disp
        
        if dx_min > dx_max or dy_min > dy_max:
            # no valid move: skip this pair
            continue
        
        # sample a valid displacement
        dx = np.random.uniform(dx_min, dx_max)
        dy = np.random.uniform(dy_min, dy_max)
        
        # apply to point i
        new_x_i = x_i + dx
        new_y_i = y_i + dy
        new_coords[i] = [new_x_i, new_y_i]
        
        # mirror to point j
        # x_j = 2*center_x - new_x_i; y_j = new_y_i
        new_coords[j,0] = 2*center_x - new_x_i
        new_coords[j,1] = new_y_i
    
    return new_coords




In [34]:
room_1 =np.array([     [0.625     , 0.        ],
                       [0.12      , 0.33      ],
                       [0.55      , 2.53      ],
                       [0.19      , 3.        ],
                       [2.31      , 3.        ],
                       [1.95      , 2.53      ],
                       [2.38      , 0.33      ],
                       [1.875     , 0.        ]])

muteated = mutate_room(room_1)
print(muteated)


[[0.625      0.        ]
 [0.12       0.33      ]
 [0.55       2.53      ]
 [0.19651728 2.98613157]
 [2.30348272 2.98613157]
 [1.95       2.53      ]
 [2.38       0.33      ]
 [1.875      0.        ]]
